In [1]:
!pip install selenium
!pip install webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 15.1 MB/s eta 0:00:00


In [2]:
!pip install openpyxl

In [9]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

In [10]:
from google.colab import files

In [11]:
import requests
from bs4 import BeautifulSoup

In [ ]:
import logging
logger = logging.getLogger("my_parser")
logger.setLevel(logging.DEBUG)

file_handler = logging.FileHandler("parser.log", mode='w', encoding='utf-8')
file_handler.setLevel(logging.DEBUG)

console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.addHandler(console_handler)

In [17]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
chrome_options.add_argument("--ignore-certificate-errors")

browser = webdriver.Chrome(options=chrome_options)

#url = 'https://moscow.drom.ru/auto/all/'
url = 'https://makhachkala.drom.ru/auto/'
links = []
cur_url = url
for i in range(100):
  browser.get(cur_url)
  time.sleep(1)

  linkElems = browser.find_elements(By.CSS_SELECTOR, 'a[data-ftid="bull_title"]')
  for l in linkElems:
    link = l.get_attribute("href")
    links.append(link)
  logger.debug(f'Полученны ссылки со страницы {i+1}')
  cur_url = f'{url}page{i+2}/'

import json

with open('links.json', 'w') as f:
  json.dump(links, f)
files.download('links.json')

2025-03-04 19:48:12,118 - my_parser - DEBUG - Полученны ссылки со страницы 1
2025-03-04 19:48:12,118 - my_parser - DEBUG - Полученны ссылки со страницы 1
DEBUG:my_parser:Полученны ссылки со страницы 1
2025-03-04 19:48:15,385 - my_parser - DEBUG - Полученны ссылки со страницы 2
2025-03-04 19:48:15,385 - my_parser - DEBUG - Полученны ссылки со страницы 2
DEBUG:my_parser:Полученны ссылки со страницы 2
2025-03-04 19:48:20,315 - my_parser - DEBUG - Полученны ссылки со страницы 3
2025-03-04 19:48:20,315 - my_parser - DEBUG - Полученны ссылки со страницы 3
DEBUG:my_parser:Полученны ссылки со страницы 3
2025-03-04 19:48:25,248 - my_parser - DEBUG - Полученны ссылки со страницы 4
2025-03-04 19:48:25,248 - my_parser - DEBUG - Полученны ссылки со страницы 4
DEBUG:my_parser:Полученны ссылки со страницы 4
2025-03-04 19:48:28,024 - my_parser - DEBUG - Полученны ссылки со страницы 5
2025-03-04 19:48:28,024 - my_parser - DEBUG - Полученны ссылки со страницы 5
DEBUG:my_parser:Полученны ссылки со страни

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
def get_info(url):
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    wait = WebDriverWait(driver, 1)

    data = {}

    try:
      name_elem = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'css-1tjirrw')))
      name_text = name_elem.text.strip()
      data['name'] = name_text[8:-19]
      data['year'] = name_text[-17:-13]

      price_elem = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'wb9m8q0')))
      data['price_rub'] = price_elem.text.replace("₽", "").replace('\xa0', '').replace(" ", "").strip()


      car_params = driver.find_elements(By.CLASS_NAME, 'css-1dzcqnh')
      car_elems = driver.find_elements(By.CLASS_NAME, 'css-1azz3as')
      for cp, ce in zip(car_params, car_elems):
        cpp = cp.text
        if 'Двигатель' in cpp:
          cee = ce.text.replace('\xa0', '').replace(" л", "").replace(" ", "").strip().split(',')
          if '.' in cee[1]:
            data['engine_type'] = cee[0]
            data['volume_liter'] = cee[1]
          else:
            data['engine_type'] = cee[1]
            data['volume_liter'] = cee[0]
        elif 'Мощность' in cpp:
          data['horsepower'] = ce.text.replace('\xa0', ' ').replace(", налог", "").replace(" л.с.", "").strip()
        elif 'Коробка передач' in cpp:
          data['transmission'] = ce.text.replace('\xa0', '').strip()
        elif 'Привод' in cpp:
          data['drive_type'] = ce.text.replace('\xa0', '').strip()
        elif 'Тип кузова' in cpp:
          data['body_type'] = ce.text.replace('\xa0', '').strip()
        elif 'Цвет' in cpp:
          data['color'] = ce.text.replace('\xa0', '').strip()
        elif 'Пробег' in cpp:
          data['mileage_km'] = ce.text.replace("км", "").replace('\xa0', '').replace(" ", "").strip()
        elif 'Руль' in cpp:
          data['wheel_side'] = ce.text.replace('\xa0', '').strip()
        #elif 'Особые отметки' in cpp:
        #  data['notes'] = ce.text.replace('\xa0', '').strip()
        elif 'Поколение' in cpp:
          data['gen'] = ce.text.replace('\xa0', '').strip()
        elif 'Комплектация' in cpp:
          data['complectation'] = ce.text.replace('\xa0', '').strip()
        else:
          continue

      text_elems = driver.find_elements(By.CLASS_NAME, 'css-pxeubi')
      if len(text_elems) > 0:
        data['post_date'] = text_elems[0].text.replace('\xa0', '').strip()[-10::]

      # text_elems = driver.find_elements(By.CLASS_NAME, 'css-1kb7l9z')
      # if len(text_elems) > 1:
      #   data['extra'] = text_elems[1].text.replace('\xa0', '').strip()

    except Exception as e:
      logger.error(f"Ошибка во время парсинга {e}", exc_info=True)
    finally:
      driver.quit()

    return data

In [21]:
import json

with open('links.json', 'r', encoding='UTF-8') as f:
  lynksy = json.load(f)

lynksy[3]

'https://auto.drom.ru/makhachkala/lada/2107/781237708.html'

In [22]:
lynksy[4]

'https://auto.drom.ru/makhachkala/lada/priora/850033253.html'

In [23]:
# ДЛЯ ТЕСТОВ

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
chrome_options.add_argument("--ignore-certificate-errors")

info = []
y = 0
for url in lynksy[4:5]:
  logger.debug(f"обработка сайта {y}")
  result = get_info(url)
  if result:
    info.append(result)
  y+=1
info

2025-03-04 19:58:57,655 - my_parser - DEBUG - обработка сайта 0
2025-03-04 19:58:57,655 - my_parser - DEBUG - обработка сайта 0
DEBUG:my_parser:обработка сайта 0


[{'name': 'Лада Приора, 2',
  'year': '1 го',
  'price_rub': '380000',
  'engine_type': 'бензин',
  'volume_liter': '1.6',
  'horsepower': '98',
  'transmission': 'механика',
  'drive_type': 'передний',
  'body_type': 'седан',
  'color': 'белый',
  'mileage_km': '120000',
  'wheel_side': 'левый',
  'gen': '1 поколение',
  'complectation': '1.6 MT Люкс 21703-02-018',
  'post_date': '05.03.2025',
  'extra': 'перейти к просмотру всех объявлений'}]

In [24]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
chrome_options.add_argument("--ignore-certificate-errors")

info = []
y = 0
for url in lynksy[730:740]:
  logger.debug(f"обработка сайта {y}")
  result = get_info(url)
  if result:
    info.append(result)
  y+=1

dt = pd.DataFrame(info)
dt.to_excel('cars_drom.xlsx', index=False)
logger.info('Данные сохранены в "cars_drom.xlsx"')
files.download('cars_drom.xlsx')

2025-03-04 19:59:11,178 - my_parser - DEBUG - обработка сайта 0
2025-03-04 19:59:11,178 - my_parser - DEBUG - обработка сайта 0
DEBUG:my_parser:обработка сайта 0
2025-03-04 19:59:15,891 - my_parser - DEBUG - обработка сайта 1
2025-03-04 19:59:15,891 - my_parser - DEBUG - обработка сайта 1
DEBUG:my_parser:обработка сайта 1
2025-03-04 19:59:21,437 - my_parser - DEBUG - обработка сайта 2
2025-03-04 19:59:21,437 - my_parser - DEBUG - обработка сайта 2
DEBUG:my_parser:обработка сайта 2
2025-03-04 19:59:27,465 - my_parser - DEBUG - обработка сайта 3
2025-03-04 19:59:27,465 - my_parser - DEBUG - обработка сайта 3
DEBUG:my_parser:обработка сайта 3
2025-03-04 19:59:32,875 - my_parser - DEBUG - обработка сайта 4
2025-03-04 19:59:32,875 - my_parser - DEBUG - обработка сайта 4
DEBUG:my_parser:обработка сайта 4
2025-03-04 19:59:38,540 - my_parser - DEBUG - обработка сайта 5
2025-03-04 19:59:38,540 - my_parser - DEBUG - обработка сайта 5
DEBUG:my_parser:обработка сайта 5
2025-03-04 19:59:42,483 - my

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>